In [3]:
import pandas as pd
import numpy as np
import json

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [4]:
l=str(np.arange(1,10,1))
f=open("data_int.txt","w")
for i in l:
    f.write(i)
!cat data_int.txt
print("\n")
m=str((np.arange(25).reshape(5,5))/10)
f1=open("data_float.txt","w")
for j in m:
    f1.write(j)
!cat data_float.txt
csv_float=pd.read_csv(r'data_float.txt')
csv_float.to_csv(r'data_float.csv',index=None)

[1 2 3 4 5 6 7 8 9]

[[0.  0.1 0.2 0.3 0.4]
 [0.5 0.6 0.7 0.8 0.9]
 [1.  1.1 1.2 1.3 1.4]
 [1.5 1.6 1.7 1.8 1.9]
 [2.  2.1 2.2 2.3 2.4]]

2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [5]:
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json
data=json.load(open('user_data.json'))
filt=[x for x in data if x['CreditCardType']=='American Express']
output=json.dumps(filt)
f=open("user_data.csv","w")
for i in output:
    f.write(i)
filt

[{'ID': '2',
  'JobTitle': 'Investment  Advisor',
  'EmailAddress': 'Clint_Thorpe5003@bulaffy.com',
  'FirstNameLastName': 'Clint Thorpe',
  'CreditCard': '7083-8766-0251-2345',
  'CreditCardType': 'American Express'},
 {'ID': '12',
  'JobTitle': 'Retail Trainee',
  'EmailAddress': 'Phillip_Carpenter9505@famism.biz',
  'FirstNameLastName': 'Phillip Carpenter',
  'CreditCard': '3657-0088-0820-5247',
  'CreditCardType': 'American Express'},
 {'ID': '28',
  'JobTitle': 'Project Manager',
  'EmailAddress': 'Russel_Graves1378@extex.org',
  'FirstNameLastName': 'Russel Graves',
  'CreditCard': '6718-4818-8011-6024',
  'CreditCardType': 'American Express'},
 {'ID': '39',
  'JobTitle': 'Stockbroker',
  'EmailAddress': 'Leanne_Newton1268@typill.biz',
  'FirstNameLastName': 'Leanne Newton',
  'CreditCard': '5438-0816-4166-4847',
  'CreditCardType': 'American Express'},
 {'ID': '57',
  'JobTitle': 'Budget Analyst',
  'EmailAddress': 'Tony_Giles1960@iatim.tech',
  'FirstNameLastName': 'Tony Giles'

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [6]:
#!wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv
file_name= "mushrooms_categorized.csv"
data = pd.read_csv(file_name)
df=str(pd.DataFrame(data).mean())
output=json.dumps(df)
print(df)
f=open("mushrooms_categorized_mean.json","w")
for i in output:
    f.write(i)

class                       0.482029
cap-shape                   3.348104
cap-surface                 1.827671
cap-color                   4.504677
bruises                     0.415559
odor                        4.144756
gill-attachment             0.974151
gill-spacing                0.161497
gill-size                   0.309207
gill-color                  4.810684
stalk-shape                 0.567208
stalk-root                  1.109798
stalk-surface-above-ring    1.575086
stalk-surface-below-ring    1.603644
stalk-color-above-ring      5.816347
stalk-color-below-ring      5.794682
veil-type                   0.000000
veil-color                  1.965534
ring-number                 1.069424
ring-type                   2.291974
spore-print-color           3.596750
population                  3.644018
habitat                     1.508616
dtype: float64


4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [106]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat
data=[]
with open('credit_card.dat','r') as a:
    for line in a:
        data.extend(map(int,line.split()))
res=""
for i in range(len(data)):
    res=str(int(str(data[i])[0:6],2))[1]+str(int(str(data[i])[6:12],2))[1]+str(int(str(data[i])[12:18],2))[1]+str(int(str(data[i])[18:24],2))[1]+" "
    res+=str(int(str(data[i])[30:36],2))[1]+str(int(str(data[i])[36:42],2))[1]+str(int(str(data[i])[42:48],2))[1]+str(int(str(data[i])[48:54],2))[1]+" "
    res+=str(int(str(data[i])[60:66],2))[1]+str(int(str(data[i])[66:72],2))[1]+str(int(str(data[i])[72:78],2))[1]+str(int(str(data[i])[78:84],2))[1]+" "
    res+=str(int(str(data[i])[90:96],2))[1]+str(int(str(data[i])[96:102],2))[1]+str(int(str(data[i])[102:108],2))[1]+str(int(str(data[i])[108:114],2))[1]
    print(res)
    

5426 3451 1553 0059
1035 6025 1132 0044
0500 8889 2899 4430
8449 1841 1520 1938
8210 9486 9240 2520
3605 0805 6563 5181
3552 6306 0865 9995
6928 9098 4130 0623
3542 9911 5189 5988
4234 9515 2904 4504
9006 6419 5160 8888
5839 8948 3152 1944
8496 1365 9418 4154
9323 3232 5222 3414
4513 1994 1080 4612
5065 3899 9325 6291
5811 0485 1106 2088
0346 3022 9652 3802
9462 0031 5358 5996
8450 0354 8353 4419
4110 6131 6565 9128
9691 1149 9953 2099
0255 4238 6628 0146
3390 1383 0341 9104
1861 5660 8409 8803
2309 3926 6823 8112
5341 1432 6591 3565
6102 0442 8254 3349
8343 0382 5946 1398
3985 3385 9545 8516
0240 9609 0226 9221
0566 8416 4649 4332
3639 3651 3252 8325
8458 9882 2891 0433
3652 3384 1826 8684
0683 3289 6240 9048
3861 6284 4198 9640
9854 9223 1891 0044
6228 2682 2622 3055
2536 4929 8464 9165
5364 8453 8193 0346
0322 9036 5210 3943
1252 3801 2212 3404
9298 8058 8212 3864
5193 2224 9982 2093
8002 5520 6188 8044
8958 0588 1923 8428
0884 0215 6832 9488
6920 2833 9554 8804
1804 5168 9029 9862


ValueError: invalid literal for int() with base 2: ''

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3

In [117]:
!wget https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

--2021-12-02 13:39:16--  https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/aamg1apjhclecka/regression_generated.csv [following]
--2021-12-02 13:39:23--  https://www.dropbox.com/s/raw/aamg1apjhclecka/regression_generated.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd1b146e210f65186cfe1c24d09.dl.dropboxusercontent.com/cd/0/inline/BbHLDiyIPwDWB1-7I1jzPvmbX8fKpVVKrDr3gr1p-V_mvlltSTYguq3OwG4sAL7t0Cr-7Mq6ey3U5kiBKczFXwP01BovqLkK7GE9iAZQS4W6AIFpwD8oPIJkj6-tOSKeo6aGXBz9uP0JSc82hoyvW8d8/file# [following]
--2021-12-02 13:39:26--  https://ucd1b146e210f65186cfe1c24d09.dl.dropboxusercontent.com/cd/0/inline/BbHLDiyIPwDWB1-7I1jzPvmbX8fKpVVKrDr3gr1p-V_mvlltSTY